In [3]:
!pip install strands-agents[mistral] python-dotenv

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18/18 [strands-agents]m [strands-agents]dk]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.


In [4]:
import json
import os
import sys
import dotenv
import boto3
import requests

import pprint

import yaml
from pathlib import Path
from typing import Dict, List, Optional, Tuple, TypeVar
from tqdm import tqdm

# Pydantic for structured data
from pydantic import BaseModel, Field

# Strands for AI agents
from strands.agent import Agent
from strands.models.mistral import MistralModel

# AWS authentication
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    load_file_content,
    get_job_paths,
    get_training_paths,
    sanity_check,
	chat_with_persona,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from src.my_utils import (
    display_markdown_file,
    call_mistral,
    get_agent,
    batch_extract,
    send_message_to_chat
)

from src.models.interview_info import(
    InterviewInfo,
    InverviewQualityInfo,
    InterviewAgentMessage
)
from src.prompts.interview_prompt import(
    INITIAL_INTERVIEW_PROMPT,
    INITIAL_INTERVIEW_PROMPT_REVERSE,
    TRANSLATE_INTERVIEW_PROMPT,
    INITIAL_INTERVIEW_QUALITY_CHECK_PROMPT
)

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

✅ API key found, we're ready to roll


In [5]:
MAX_TURNS_IN_INTERVIEW = 10

In [6]:
DATA_JOBS_DIR = Path('../data_jobs')
DATA_TRAININGS_DIR = Path('../data_trainings')
DATA_INTERVIEWS_DIR = Path('../data_interviews')

In [7]:
with open("../src/config.yaml", "r") as f:
    config = yaml.safe_load(f)

job_data_version = config["job_data_version"]
print(f"job_data_version version: {job_data_version}")

training_data_version = config["training_data_version"]
print(f"training_data_version version: {training_data_version}")

interview_data_version = config["interview_data_version"]
print(f"interview_data_version version: {interview_data_version}")

job_data_version version: v4
training_data_version version: v7
interview_data_version version: v8


In [8]:
filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
jobs_map = read_json(save_path)

In [9]:
# Load skills domains data
filename = f"final_map_clusters_trainings_{training_data_version}.json"
save_path = DATA_TRAININGS_DIR / filename
trainings_map = read_json(save_path)

print(f"✅ Loaded {len(trainings_map)} skills domains")
print("\n" + "="*50)

✅ Loaded 12 skills domains



# Perform interviews

In [10]:
def conduct_persona_interview(
    persona_id: str,
    max_turns: int = 5,
    model: str = "mistral-medium-latest",
    print_conversation: bool = False
) -> List[str]:
    """Interview a persona and return conversation transcript"""

    interview = InterviewInfo()

    # prompt = INITIAL_INTERVIEW_PROMPT
    prompt = INITIAL_INTERVIEW_PROMPT_REVERSE

    interview_agent = get_agent(prompt, model_id=model)
    conversation_id = None

    # Start with greeting
    agent_message = "Hello, I'm here to help you navigate your career path.\n"
    #agent_message += "To do so effectively, I may need to collect some personal information — rest assured, your privacy is fully respected and protected."
    if print_conversation:
        print(f"Assistant: {agent_message}")
    interview.interview.append(f"Assistant: {agent_message}")

    # Conduct interview
    for turn in range(max_turns):
        
        resp = send_message_to_chat(agent_message, persona_id, conversation_id)
        print(resp)

        if resp is None:
            break

        user_response, conversation_id = resp
        interview.conversation_id = conversation_id
        interview.interview.append(f"User: {user_response}")
        if print_conversation:
            print(f"User: {user_response}")

        # Generate next question
        conversation_str = '\n'.join(interview.interview)
        agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)
        # agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, user_response)

        # Track cost (using utils.py function)
        # track_api_call(agent_response, model)

        if agent_response.conversation_finished is True:
            break
            
        agent_message = agent_response.message
        interview.interview.append(f"Assistant: {agent_message}")
        if print_conversation:
            print(f"Assistant: {agent_message}")

    return interview

In [9]:
# Interview all personas
#persona_ids = [f'persona_{i:03}' for i in range(1, 4)]
persona_ids = [f'persona_{i:03}' for i in range(1, 101)]

# personas_save_path = SUBMISSION_DIR / 'personas.json'
filename = f"interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename

if not interviews_save_path.exists():
    save_json(interviews_save_path, {})

interviews = read_json(interviews_save_path)
personas_to_process = len(persona_ids) - len(interviews)
print(f'Personas to process: {personas_to_process}')

# Reset cost tracker if starting fresh
if len(interviews) == 0:
    reset_cost_tracker()
    print("💰 Starting fresh - cost tracker reset")

# Track how many new personas we process
new_personas_processed = 0

for persona_id in tqdm(persona_ids):
    if persona_id in interviews:
        interview = interviews[persona_id]['interview']
        if len(interview) > 1:
            continue
        
    new_personas_processed += 1

    activity_domains_str = ""
    first = True
    for domain in jobs_map:
        if first is False:
            activity_domains_str += ", "
        activity_domains_str += domain
        first = False
        
    skill_domains_str = ""
    first = True
    for domain in trainings_map:
        if first is False:
            skill_domains_str += ", "
        skill_domains_str += domain
        first = False
    
    # Interview
    conversation = conduct_persona_interview(persona_id, max_turns=MAX_TURNS_IN_INTERVIEW, print_conversation=False)
    interviews[persona_id] = conversation.model_dump()

    # Save every personas
    if len(interviews) % 1 == 0:
        save_json(interviews_save_path, interviews)

    # Show cost update every 20 personas
    # if new_personas_processed > 0 and new_personas_processed % 20 == 0:
    #     print(f"\n💰 Cost update after {new_personas_processed} new personas:")
    #     print_cost_summary()
    #     print()

    # if new_personas_processed > 4:break

save_json(interviews_save_path, interviews)

print(f"\n✅ Interviewed {len(interviews)} personas total ({new_personas_processed} new)")

# Final cost summary for persona processing
if new_personas_processed > 0:
    print("\n📊 Persona processing costs:")
    print_cost_summary()

Personas to process: 0


  0%|          | 0/100 [00:00<?, ?it/s]

('Oh, hello. I don’t have much experience, but I’m trying to find a job in Industrial HSE. Maybe you can help?', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_084#2025-10-20T19:38:53')
('I’m open to relocate, even abroad. I just want to work in compliance reporting or hazardous materials... but I’m still learning.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_084#2025-10-20T19:38:53')
('I’m 20. No country in mind—I’ll go where there’s a job. I don’t want to say where I live now, but I can move. Sorry if that’s not helpful.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_084#2025-10-20T19:38:53')


 84%|████████▍ | 84/100 [00:14<00:02,  5.72it/s]

('Hello! I am Lucas, 16 years old. I like math and talking with people. Can you tell me about jobs in HSE? Do they use math a lot? Or do they talk more? Sorry my English, I am still learning.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_085#2025-10-20T19:39:08')
('Oh, I live in Salvador, Brazil! I not looking for job, just want know things. Like... do HSE people count numbers every day? Or they write reports and talk to workers? I like math but also like talk to people. Is HSE good for that? I dont want move from Salvador.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_085#2025-10-20T19:39:08')
('Yes, I go to school still, Ensino Fundamental. I just want know about HSE jobs in Brazil, in Salvador. Like... if they use math to check if machines are safe? Or if they talk to workers to teach safety? I curious how math and talking mix in this job. You can tell me?', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_085#2025-10-20T19:39:08')


...

('Oh, that sounds amazing! I’ve always wanted a life full of adventure, you know? Something where every day feels different, where I can explore and see new things. I just don’t know where to start, but I’m so ready for it. What kind of exciting paths are out there?', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_099#2025-10-20T19:42:17')
('I’m 28 and definitely not under 16—haha! I’m not really looking for a job just yet, but I’m super curious about training opportunities. Something that prepares me for an exciting life, where I can move around, see new places, and maybe even work in different environments. I just want to feel ready for whatever adventure comes my way! What kind of trainings could open doors like that?', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_099#2025-10-20T19:42:17')
('I’m in Salvador—it’s such a vibrant place, full of energy! I love that there’s always something happening here, but I’m hungry for even more. More movement, more discovery. I’m not worried about

 99%|█████████▉| 99/100 [03:41<00:15, 15.59s/it]

('Oh, that sounds amazing! I’ve always dreamed of a life full of adventure and travel. The sea calls to me, you know? I want to be out there, exploring the world, not stuck in one place.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_100#2025-10-20T19:42:37')
('Oh, I’m in Rio—it’s where my heart is, with the ocean right there! I’m not really looking for a job yet, but I want to learn, to prepare for something big. Trainings sound perfect, especially if they can teach me things for my future adventures. I just know I don’t want to be tied to land forever!', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_100#2025-10-20T19:42:37')
('Oh, age is just a number when you have big dreams, right? I’m still young, but old enough to know what I want—freedom, the sea, and a life full of stories to tell! Let’s just say I’m at the perfect age to start learning for my future.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_100#2025-10-20T19:42:37')
('Oh, I don’t want to go too far from Rio yet—I love 

100%|██████████| 100/100 [03:58<00:00,  2.39s/it]


✅ Interviewed 100 personas total (17 new)

📊 Persona processing costs:
💰 Cost Summary:
  Total API calls: 0
  Total tokens: 0
  Estimated cost: $0.0000


---

# For Debug Only

---

# Check interview qualities

In [10]:
def check_interview_quality(
    interview: str,
    model: str = "mistral-small-latest",
    print_prompt: bool = False
) -> InverviewQualityInfo:
    prompt = INITIAL_INTERVIEW_QUALITY_CHECK_PROMPT.format(
        interview=interview
    )

    if print_prompt is True:
        print(prompt)

    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=InverviewQualityInfo, prompt=prompt)

    return result

In [11]:
filename = f"interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)

In [12]:
cache_period = 5

filename = f"quality_interviews_{interview_data_version}.json"
save_path = DATA_INTERVIEWS_DIR / filename
if not save_path.exists():
    save_json(save_path, {})
quality_interviews = read_json(save_path)


new_items_processed = 0
for persona_id in tqdm(interviews):
    if persona_id in quality_interviews:
        quality = json.loads(quality_interviews[persona_id])
        if quality['quality_level'] == 'OK':
            continue

    new_items_processed = new_items_processed + 1

    interview = interviews[persona_id]['interview']

    interview_str = "\n".join(interview)
    # print(interview_str)

    quality = check_interview_quality(interview_str, print_prompt=False)
    quality_str = json.dumps(quality.model_dump(), ensure_ascii=False)

    quality_interviews[persona_id] = quality_str

    if new_items_processed % cache_period == 0:
        save_json(save_path, quality_interviews)

    # if new_items_processed > 0:break

save_json(save_path, quality_interviews)

100%|██████████| 100/100 [01:57<00:00,  1.17s/it]


In [13]:
filename = f"quality_interviews_{interview_data_version}.json"
save_path = DATA_INTERVIEWS_DIR / filename
quality_interviews = read_json(save_path)

for persona_id in interviews:
    quality = json.loads(quality_interviews[persona_id])
    if quality['quality_level'] != 'OK':
        print(persona_id)
        print(quality['rationale'])
        print('---')

persona_007
The interview does not provide clear information about the user's location (city) and their specific interests in jobs or training. The user's responses are poetic and metaphorical, making it difficult to extract concrete information. The assistant also did not receive a direct answer to the question about the city where the user lives or specific areas of interest for jobs or training.
---
persona_012
The interview is missing the location information. The user mentioned they are in Rio, but it is not clear if this is Rio de Janeiro in Brazil or another city named Rio. Additionally, the user's age is below 16, making other information non-mandatory, but the location ambiguity still affects the quality level.
---


# Redo interview of a Persona

In [17]:
persona_id = "persona_025"

# Interview
print("🎤 Conduct Interview...")
conversation = conduct_persona_interview(persona_id, max_turns=10, print_conversation=True)

filename = f"interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)
interviews[persona_id] = conversation.model_dump()
save_json(interviews_save_path, interviews)

filename = f"full_interviews_{persona_id}.json"
full_interview_save_path = DATA_INTERVIEWS_DIR / filename
full_interview = conversation.model_dump()
save_json(full_interview_save_path, full_interview)

🎤 Conduct Interview...
Assistant: Hello, I'm here to help you navigate your career path.

('Hi! I like drawing and playing video games, especially ones with cool machines. Sometimes I watch videos of big factories too.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_025#2025-10-30T20:26:15')
User: Hi! I like drawing and playing video games, especially ones with cool machines. Sometimes I watch videos of big factories too.


/tmp/ipykernel_11426/777025835.py:41: DeprecationWarning: Agent.structured_output method is deprecated. You should pass in `structured_output_model` directly into the agent invocation. see: https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/structured-output/
  agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)


Assistant: Do you want to find a job in Brazil, explore training options, or simply gather information? Also, are you under 16 years old?
("I'm only 14, so no jobs yet! But I'd love to find a training where I can learn about real machines, not just the ones in games. Maybe something fun and hands-on?", '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_025#2025-10-30T20:26:15')
User: I'm only 14, so no jobs yet! But I'd love to find a training where I can learn about real machines, not just the ones in games. Maybe something fun and hands-on?
Assistant: Thanks for sharing! To assist you better, could you please tell me which city you currently live in?
('I live in Brasília! I don’t wanna move, but I’d love to find a training here. Maybe something where I can see how machines work for real?', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_025#2025-10-30T20:26:15')
User: I live in Brasília! I don’t wanna move, but I’d love to find a training here. Maybe something where I can see how machines w

In [12]:
filename = f"interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)
interviews[persona_id] = conversation.model_dump()
save_json(interviews_save_path, interviews)

# Debug

In [ ]:
if False:
    interview = conduct_persona_interview(
        'persona_079',
        max_turns=5,
        model="mistral-small-latest",
        print_conversation=True
    )
    
    print(interview)

# Translate interviews in english

In [ ]:
def translate_interview(
    interview,
    model: str = "mistral-small-latest",
    print_prompt=False
) -> InterviewInfo:

    prompt = TRANSLATE_INTERVIEW_PROMPT.format(
        interview=interview
    )

    if print_prompt is True:
        print(prompt)

    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=InterviewInfo, prompt=prompt)

    return result

In [ ]:
filename = f"interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)

In [ ]:
cache_period = 5

filename = f"en_interviews_{interview_data_version}.json"
save_path = DATA_INTERVIEWS_DIR / filename
if not save_path.exists():
    save_json(save_path, {})
en_interviews = read_json(save_path)


new_items_processed = 0
for interview_id in tqdm(interviews):
    if interview_id in en_interviews:
        continue

    new_items_processed = new_items_processed + 1
    
    interview = interviews[interview_id]

    translated_interview = translate_interview(interview, print_prompt=False)
    en_interviews[interview_id] = translated_interview.interview

    if new_items_processed % cache_period == 0:
        save_json(save_path, en_interviews)

    # if new_items_processed > 4:break

save_json(save_path, en_interviews)